# Import libraries

In [ ]:
!pip install datasets
!pip install evaluate rouge_score
!pip install accelerate

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, Dataset, DatasetDict
import evaluate
import numpy as np
import pandas as pd
import accelerate

# Model

In [3]:
model_path ="VietAI/vit5-base"
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model, return_tensors = "pt")

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# Dataset

In [4]:
data = load_dataset("Valleyy/final_nlp_data")

In [5]:
def data_processing(dataset):
    document = dataset["Segmented_document"]
    summary = dataset["Segmented_summary"]
    doc_tokenizer = tokenizer(document, max_length = 1024, truncation = True)
    sum_tokenizer = tokenizer(summary, max_length = 1024, truncation = True)
    input_ids = doc_tokenizer["input_ids"]
    attention_mask = doc_tokenizer["attention_mask"]
    labels = sum_tokenizer["input_ids"]
    dataset = DatasetDict({"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels})
    return dataset

In [ ]:
dataset = data.map(data_processing, remove_columns=[ 'Document', 'Summary', 'Segmented_document','Segmented_summary', 'Dataset'], batched = True, batch_size = 128)

# Training

In [8]:
metric = evaluate.load("rouge")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions[predictions == -100] = 1
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.eos_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True,)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
args = Seq2SeqTrainingArguments(output_dir = "/kaggle/working/",
                                evaluation_strategy="epoch",
                                save_strategy="epoch",
                               per_device_train_batch_size=3,
                               per_device_eval_batch_size=3,
                               learning_rate=1e-4,
                               weight_decay=1e-2,
                               load_best_model_at_end =True,
                               predict_with_generate=True,
                               num_train_epochs=5,
                               logging_strategy="epoch",
                               generation_max_length=256,
                                save_total_limit = 1,
                               fp16=True)

trainer = Seq2SeqTrainer(model=model,
                        args=args,
                        train_dataset=dataset["train"],
                        eval_dataset=dataset["valid"],
                        tokenizer=tokenizer,
                        compute_metrics=compute_metrics,
                        data_collator = data_collator
                        )

In [10]:
torch.cuda.empty_cache()
torch.cuda.memory_allocated()
PYTORCH_CUDA_ALLOC_CONF=expandable_segments=256

In [ ]:
trainer.train()

In [ ]:
import os

# Path to the directory to save the model
save_model_dir = '/kaggle/working/save_model'

# Check if the directory exists, if not, create it
if not os.path.exists(save_model_dir):
    os.makedirs(save_model_dir)
    print(f"The directory '{save_model_dir}' has been created.")
else:
    print(f"The directory '{save_model_dir}' already exists.")

In [ ]:
trainer.save_model("/kaggle/working/save_model")